In [1]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

You should consider upgrading via the 'c:\users\ahmed sayed mansour\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.



  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14950 sha256=3019f92c6606d414e5de5ffe75e12e7503d49808a72dd5aba42055e1f679eeb8
  Stored in directory: C:\Users\Ahmed Sayed Mansour\AppData\Local\Temp\pip-ephem-wheel-cache-289ammyk\wheels\c0\9f\1a\0d06f83f42c9aa212ea12a9008799a134ec3f1616c73cf8006
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30144 sha256=b6f7afb15002b0dc900cb9c9a531c2c6aae0b05ea5dd8300441525b6beec157e
  Stored in directory: C:\Users\Ahmed Sayed Mansour\AppData\Local\Temp\pip-ephem-wheel-cache-289ammyk\wheels\32\9a\77\d91ff190ee49f0e3bf767d20c04b3d286015ae166070e270e3
Successfully built hpacellseg pytorch-zoo


In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

In [4]:
# Input: list of image filters as png
# Output: list of image filters as np.arrays
def image_to_arrays(path):
    
    image_arrays = list()
    for image in path:
        array = np.asarray(Image.open(image))
        image_arrays.append(array)
        
    return image_arrays

In [5]:
# Get single image that blends all RGBY into RGB
# Introduce the images as arrays. Can use the function above.

def get_blended_image(images): 
    # get rgby images for sample

    # blend rgby images into single array
    blended_array = np.stack(images[:-1], 2)

    # Create PIL Image
    blended_image = Image.fromarray( np.uint8(blended_array) )
    return blended_image

In [6]:
# Introduce list of image filters
# Returns a processed image ready for the CNN and an encoded label as tensor
def image_prep(paths, label):

    img = image_to_arrays(paths)
    size = np.shape(img[0])[0]
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.reshape(img, (1, size, size, 3))
    img = tf.image.resize(img, IMG_SIZE)

    label = tf.strings.split(label, sep='|')
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.reduce_sum(tf.one_hot(indices=label, depth=19), axis=0)
    label = tf.reshape(label, (1, 19))
    
    return img, label

In [7]:
def apply_augmentation(image, label):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((IMG_SIZE[0], IMG_SIZE[0], 3))
    
    return aug_img, label

In [8]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [11]:
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"

train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))

In [21]:
colours = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
#TRAIN = 'data/train'
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

In [ ]:
# Let's check out the label distribution frequency.
label_counts = []
for label in train['Label']:
    sep = label.split('|')
    for num in sep:
        labels.append(int(num))
counts = pd.value_counts(labels)

# It's an ugly plot, but I'm trying to save some time here...
plt.bar(x = counts.index,height=counts)
plt.xticks(counts.index)
plt.show()

In [ ]:
titles = ['microtubules', 'nuclei', 'endoplasmic reticulum', 'protein of interest']
fig, axs = plt.subplots(3, 4, figsize =(16,8))
for entry in range(3):
    for channel in range(4):
        img = plt.imread(paths[entry][channel])
        axs[entry, channel].imshow(img)        
        if entry == 0:
            axs[0, channel].set_title(titles[channel])

In [ ]:
NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=False,
    multi_channel_model=True,
)

image = paths[4]
arrays = image_to_arrays(image)
nuclei = arrays[1]
cell = arrays[:-1]

# Nuclei segmentation
nuc_segmentations = segmentator.pred_nuclei([nuclei])

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(arrays[1])
ax[0].set_title('Original Nucleis', size=20)
ax[1].imshow(nuc_segmentations[0])
ax[1].set_title('Segmented Nucleis', size=20)
plt.show()

# Cell segmentation
inter_step = [[i] for i in image[:-1]]
cell_segmentations = segmentator.pred_cells(inter_step)

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(get_blended_image(arrays))
ax[0].set_title('Original Cells', size=20)
ax[1].imshow(cell_segmentations[0])
ax[1].set_title('Segmented Cells', size=20)
plt.show()

In [ ]:
# Nuclei mask
nuclei_mask = label_nuclei(nuc_segmentations[0])
# Cell masks
cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
# Plotting
f, ax = plt.subplots(1, 3, figsize=(16,16))
ax[0].imshow(nuclei_mask)
ax[0].set_title('Nuclei Mask', size=20)
ax[1].imshow(cell_nuclei_mask)
ax[1].set_title('Cell Nuclei Mask', size=20)
ax[2].imshow(cell_mask)
ax[2].set_title('Cell Mask', size=20)
plt.show()

In [ ]:
# Let's stack the original image and the segmentation mask, to see how the segmentation worked out
plt.figure(figsize=(20,20))
plt.imshow(get_blended_image(arrays))
plt.imshow(cell_mask, alpha=0.5)
plt.title('Segmentation results', size=40)
plt.axis('off')
plt.show()

In [ ]:
# Unique vector of cell_mask numbers
numbers = set(np.ravel(cell_mask))
numbers.remove(0)

fig = plt.figure(figsize=(25,6*len(numbers)/4))
index = 1

ax = fig.add_subplot(len(numbers)//4+1, 4, index)
ax.set_title("Complete Cell Mask", size=20)
plt.imshow(cell_mask)

index += 1
for number in numbers:
    isolated_cell = np.where(cell_mask==number, cell_mask, 0)
    ax = fig.add_subplot(len(numbers)//4+1, 4, index)
    ax.set_title("Segment {number}", size=20)
    plt.imshow(isolated_cell)
    index += 1

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
import wandb

In [ ]:
LABELS= {
0: "Nucleoplasm",
1: "Nuclear membrane",
2: "Nucleoli",
3: "Nucleoli fibrillar center",
4: "Nuclear speckles",
5: "Nuclear bodies",
6: "Endoplasmic reticulum",
7: "Golgi apparatus",
8: "Intermediate filaments",
9: "Actin filaments",
10: "Microtubules",
11: "Mitotic spindle",
12: "Centrosome",
13: "Plasma membrane",
14: "Mitochondria",
15: "Aggresome",
16: "Cytosol",
17: "Vesicles and punctate cytosolic patterns",
18: "Negative"
}

In [ ]:
IMG_SIZE = [224, 224]
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

colours = ['_red.png', '_blue.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

In [24]:
datasize = 1000
trainsize = 700
valsize = 150
testsize = 150

In [ ]:
# Processing the data for training:
training_data = []
for i,path in enumerate(paths[:trainsize]):
    img, label = image_prep(path, train['Label'][i])
    training_data.append([img,label])

train_ds = tf.data.Dataset.from_tensor_slices(([training_data[i][0] for i in range(len(training_data))], [training_data[i][1] for i in range(len(training_data))]))
len(train_ds)

In [ ]:
#validation data
val_data = []
start_img = trainsize
val_num = valsize
for i,path in enumerate(paths[start_img:start_img+val_num]):
    img, label = image_prep(path, train['Label'][i+start_img])
    val_data.append([img,label])

val_ds = tf.data.Dataset.from_tensor_slices(([val_data[i][0] for i in range(len(val_data))], [val_data[i][1] for i in range(len(val_data))]))
len(val_ds)

In [ ]:
#test data
test_data = []
start_img_test = trainsize + valsize
test_num = testsize
for i,path in enumerate(paths[start_img_test:start_img_test+test_num]):
    img, label = image_prep(path, train['Label'][i+start_img_test])
    test_data.append([img,label])

test_ds = tf.data.Dataset.from_tensor_slices(([test_data[i][0] for i in range(len(test_data))], [test_data[i][1] for i in range(len(test_data))]))
len(test_ds)

In [ ]:
base_model = DenseNet121(include_top=False, weights='imagenet')
base_model.trainable = True

inputs = layers.Input((IMG_SIZE[0], IMG_SIZE[0], 3))

x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(LABELS), activation='sigmoid')(x)

tf.keras.backend.clear_session()

model = Model(inputs, outputs)
model.summary()

In [ ]:
tf.keras.backend.clear_session()

earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=0, mode='min',
    restore_best_weights=True
)

model.compile('adam', 'binary_crossentropy', metrics=[tf.keras.metrics.AUC(multi_label=True)])
#model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
#run = wandb.init(entity='ayush-thakur', project='hpa', job_type='train')

hist = model.fit(train_ds, 
          epochs=10,
          validation_data=val_ds,
          verbose=1,
          callbacks=[earlystopper]
                )
#plot_hist(hist)
#run.finish()

In [ ]:
probs_densenet = model.predict(test_ds, verbose=1)

In [ ]:
test_labels = []
test_labels_pre = []
for i in range (testsize) :
    test_labels.append(np.argmax(test_data[i][1]))
    test_labels_pre.append(np.argmax(probs_densenet[0]))

In [ ]:
print('train percentage :', (trainsize/datasize)*100)
print('validation percentage :', (valsize/datasize)*100)
print('test percentage :', (testsize/datasize)*100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, test_labels_pre))